# ETL (Extrair, Transformar e Carregar) da camada Raw para Silver
Este notebook tem como objetivo aplicar, em nível moderado, a transformação e limpeza dos dados brutos da camada Raw para a camada Silver, a fim de fornecer uma “visão corporativa” da principal entidade do negócio. Ele processa os dados brutos do banco de dados da Comunicação de Acidente de Trabalho (CAT) do INSS.

## 1. Extração
Seção de extração dos dados brutos do arquivo CSV



In [1]:
!pip install pyspark
from pyspark.sql import SparkSession


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
spark = SparkSession.builder \
    .appName("CSVtoPostgresETL") \
    .config("spark.jars", "path/to/postgresql-42.x.x.jar") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("OFF")

df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("../Data layer/raw/dados_brutos.csv")

df.head()
df.printSchema()


26/01/12 18:29:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/12 18:29:34 WARN DependencyUtils: Local jar /home/yabamiah/Sandbox/cat-analytics/Transformer/path/to/postgresql-42.x.x.jar does not exist, skipping.
26/01/12 18:29:34 INFO SparkContext: Running Spark version 4.1.1
26/01/12 18:29:34 INFO SparkContext: OS info Linux, 6.17.12-300.fc43.x86_64, amd64
26/01/12 18:29:34 INFO SparkContext: Java version 17.0.17+10
26/01/12 18:29:34 INFO ResourceUtils: ==============================================================
26/01/12 18:29:34 INFO ResourceUtils: No custom resources configured for spark.driver.
26/01/12 18:29:34 INFO ResourceUtils: ==============================================================
26/01/12 18:29:34 INFO SparkContext: Submitted application: CSVtoPostgresETL
26/01/12 18:29:34 INFO SecurityManager: Changi

root
 |-- Agente  Causador  Acidente: string (nullable = true)
 |-- Data Acidente1: string (nullable = true)
 |-- CBO: string (nullable = true)
 |-- CID-10: string (nullable = true)
 |-- CNAE2.0 Empregador: integer (nullable = true)
 |-- CNAE2.0 Empregador Descrição: string (nullable = true)
 |-- Emitente CAT: string (nullable = true)
 |-- Espécie do benefício: string (nullable = true)
 |-- Filiação Segurado: string (nullable = true)
 |-- Indica Óbito Acidente: string (nullable = true)
 |-- Munic Empr: string (nullable = true)
 |-- Natureza da Lesão: string (nullable = true)
 |-- Origem de Cadastramento CAT: string (nullable = true)
 |-- Parte Corpo Atingida: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo do Acidente: string (nullable = true)
 |-- UF  Munic.  Acidente: string (nullable = true)
 |-- UF Munic. Empregador: string (nullable = true)
 |-- Data Acidente18: string (nullable = true)
 |-- Data Despacho Benefício: string (nullable = true)
 |-- Data Acident

## 2. Transformação
Nesta seção, iremos transformar, tratar e limpar os dados vindos da camada Raw para a camada Silver, ainda utilizando o Spark.

### 2.1 Importações e configuração
Aqui iremos importar o necessário do pyspark, definir algumas variáveis para auxiliar nas transformações.

In [3]:
from pyspark.sql.functions import col, count, when, sum as spark_sum
from pyspark.sql.types import StringType

nao_classif_string = '{ñ class}'
nao_classif_numeric = 0

limite_repeticao = 0.95
limite_nulos = 0.95

total_linhas = df.count()
colunas_a_remover = []
novas_colunas = []

### 2.2 Transformando Não Classificados para Nulos
Fazendo a transformação de valores não classificados para nulos

In [4]:
for coluna in df.columns:
    tipo_coluna = df.schema[coluna].dataType

    # Fazemos isso pois algumas colunas possuem '.', o que buga
    coluna_escaped = col(f"`{coluna}`")

    if isinstance(tipo_coluna, StringType):
        expressao = when(
            coluna_escaped == nao_classif_string, None
        ).otherwise(coluna_escaped).alias(coluna)
    else:
        expressao = when(coluna_escaped == nao_classif_numeric, None
        ).otherwise(coluna_escaped).alias(coluna)

    novas_colunas.append(expressao)

# Executando as expressões de trocar dos valores não classificados para nulos
df_limpo = df.select(*novas_colunas)

### 2.3 Remoção de Colunas com Dados Não Classificados ou Constante
Aqui iremos remover as colunas que não agregam valor para a análise. Sendo as características utilizadas para as colunas caírem nesse termo são:

- 95%+ dos valores são iguais
- 60%+ dos valores são nulos/não classificados
- Variância = 0
- Não agrega informação

In [5]:
for coluna in df_limpo.columns:
    # Fazemos isso pois algumas colunas possuem '.', o que buga
    coluna_escaped = f"`{coluna}`"

    # 1. Verifica Porcentagem de Nulos
    # Como já retiramos os não classificados, agora só contar os nulos diretamente
    qtd_nulos = df_limpo.select(count(when(col(coluna_escaped).isNull(), 1))).collect()[0][0]

    porcentagem_nulos = (qtd_nulos / total_linhas) * 100

    if porcentagem_nulos > (limite_nulos * 100):
        print(f"Removendo {coluna}: {porcentagem_nulos:.2f}% de nulos (incluindo não classificados convertidos).")
        colunas_a_remover.append(coluna)
        continue

    # 2. Verifica valores repetitivos (>95%)
    repeticao_maxima = df_limpo.groupBy(coluna_escaped).count() \
        .agg({"count": "max"}).collect()[0][0]

    if repeticao_maxima is None: repeticao_maxima = 0

    porcentagem_maxima = (repeticao_maxima / total_linhas) * 100

    if porcentagem_maxima > (limite_repeticao * 100):
        print(f"Removendo {coluna}: {porcentagem_maxima:.2f}% de valor repetido.")
        colunas_a_remover.append(coluna)

if colunas_a_remover:
    df_final = df_limpo.drop(*colunas_a_remover)
    print(f"\nTotal removido: {len(colunas_a_remover)}")
else:
    df_final = df_limpo
    print("\nNenhuma coluna removida.")

df_limpo.unpersist()
df_final.head()

Removendo Emitente CAT: 95.82% de valor repetido.
Removendo Espécie do benefício: 100.00% de valor repetido.
Removendo Filiação Segurado: 99.13% de valor repetido.
Removendo Indica Óbito Acidente: 99.54% de valor repetido.
Removendo Origem de Cadastramento CAT: 100.00% de valor repetido.
Removendo Data Despacho Benefício: 100.00% de valor repetido.


Removendo CNPJ/CEI Empregador: 96.25% de valor repetido.

Total removido: 7


Row(Agente  Causador  Acidente='Ataque de Ser Vivo por Mordedura, Picada, Chi', Data Acidente1='2023/04', CBO='621005-Trab. Agropecuário em Geral', CID-10='M23.8 Outr Transt Internos do Joelho', CNAE2.0 Empregador=None, CNAE2.0 Empregador Descrição=None, Munic Empr='000000-Ignorado', Natureza da Lesão='Lesao Imediata', Parte Corpo Atingida='Joelho', Sexo='Masculino', Tipo do Acidente='Típico', UF  Munic.  Acidente='Zerado', UF Munic. Empregador='Zerado', Data Acidente18='2023/04', Data Acidente20='24/04/2023', Data Nascimento='05/06/1977', Data Emissão CAT='01/05/2023')

### 2.4 Remoção de Colunas Altamente Correlacionada
Serão identificadas e removidas colunas que possuem conteúdo idêntico ou derivado direto de outras colunas existentes, eliminando redundâncias desnecessárias.

In [6]:
import re

def tirar_sufixo_numerico(coluna):
  return re.sub(r'\d+$', '', coluna)  # Apenas sufixo

colunas_a_remover = []
colunas_limpas = set()

for coluna in df.columns:
  nome_limpo = tirar_sufixo_numerico(coluna).lower()
  print(nome_limpo)

  if nome_limpo in colunas_limpas:
    print("Removendo coluna: ", nome_limpo)
    colunas_a_remover.append(coluna)
  else:
    colunas_limpas.add(nome_limpo)

df_final = df_final.drop(*colunas_a_remover)
df_final.head()

agente  causador  acidente
data acidente
cbo
cid-
cnae2.0 empregador
cnae2.0 empregador descrição
emitente cat
espécie do benefício
filiação segurado
indica óbito acidente
munic empr
natureza da lesão
origem de cadastramento cat
parte corpo atingida
sexo
tipo do acidente
uf  munic.  acidente
uf munic. empregador
data acidente
Removendo coluna:  data acidente
data despacho benefício
data acidente
Removendo coluna:  data acidente
data nascimento
data emissão cat
cnpj/cei empregador


Row(Agente  Causador  Acidente='Ataque de Ser Vivo por Mordedura, Picada, Chi', Data Acidente1='2023/04', CBO='621005-Trab. Agropecuário em Geral', CID-10='M23.8 Outr Transt Internos do Joelho', CNAE2.0 Empregador=None, CNAE2.0 Empregador Descrição=None, Munic Empr='000000-Ignorado', Natureza da Lesão='Lesao Imediata', Parte Corpo Atingida='Joelho', Sexo='Masculino', Tipo do Acidente='Típico', UF  Munic.  Acidente='Zerado', UF Munic. Empregador='Zerado', Data Nascimento='05/06/1977', Data Emissão CAT='01/05/2023')

### 2.5 Tratando/Limpando valores nulos

O tratamento dos campos nulos foi definido com base no impacto analítico de cada variável:

(Agente Causador, CBO e UF. Munic. do Acidente): Valores nulos serão alterados para o termo "Não identificado". Isso preserva o registro para contagem total, já que outras variáveis permitem análises parciais

(CID e CNAE): São campos essenciais. Sem o CID, perde-se a causa médica; sem o CNAE, inviabiliza-se a análise por setor econômicos. Se nulos, serão descartados, pois a falta desses dados inviabiliza a análise.


In [7]:
cols_para_preencher = ["Agente  Causador  Acidente", "CBO", "`UF  Munic.  Acidente`"]
cols_criticas_para_drop = ["CID-10", "`CNAE2.0 Empregador`", "`CNAE2.0 Empregador Descrição`"]

df_tratado = df_final \
    .na.fill("Não identificado", subset=cols_para_preencher) \
    .na.drop(subset=cols_criticas_para_drop)

print(f"Total antes: {df_final.count()}")
print(f"Total depois: {df_tratado.count()}")

df_tratado.head()

Total antes: 157845
Total depois: 145692


Row(Agente  Causador  Acidente='Rua e Estrada - Superficie Utilizada para Sus', Data Acidente1='2023/03', CBO='252105-Administrador', CID-10='S42.0 Frat da Clavicula', CNAE2.0 Empregador=4711, CNAE2.0 Empregador Descrição='Comercio Varejista de Mercadorias em Geral, c', Munic Empr='354990-São José dos Campos', Natureza da Lesão='Fratura', Parte Corpo Atingida='Ombro', Sexo='Masculino', Tipo do Acidente='Trajeto', UF  Munic.  Acidente='Maranhão', UF Munic. Empregador='São Paulo', Data Nascimento='05/06/1977', Data Emissão CAT='01/05/2023')

## 3. Carregamento